In [ ]:
from seq_loader.names_loader import TDTSDataset, collate_padded_batch_fn
from seq_loader.lightning import LitLSTM
from pathlib import Path
import torch
import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
data_path = Path('/home/jovyan/data/tbts_data/')
filename = "tbts-names"
_NAME_CHARS_COL = str("name_chars")
tdts_dataset = TDTSDataset(
    tbts_path=data_path / f"{filename}.parquet",
    tbts_groupby_column="G",
    cat_columns=[_NAME_CHARS_COL],
    labels_path=data_path / f"{filename}-labels.parquet",
)

In [ ]:
embedding_dim = 4
col_embedding_dims = dict.fromkeys(tdts_dataset.cat_columns, embedding_dim)

In [ ]:
embedding_dims = tdts_dataset.cat_col_embeddings_params(col_embedding_dims=col_embedding_dims)

In [ ]:
model = LitLSTM(embedding_dims=embedding_dims, h_size=8, n_classes=tdts_dataset.n_calsses, lr=1e-3)

In [ ]:
model

In [ ]:
loader = torch.utils.data.DataLoader(
        dataset=tdts_dataset,
        shuffle=True,
        batch_size=10,
        collate_fn=collate_padded_batch_fn,
    )

In [ ]:
logger = TensorBoardLogger("tb_logs", name="last_names")

In [ ]:
trainer = L.Trainer(accelerator="cuda", max_epochs=200, logger=logger)

In [ ]:
trainer.fit(model=model, train_dataloaders=loader)

In [ ]:
! dirname $data_path